# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [28]:
def priest(sex, age, respiratoryRate, oxygenSaturation, heartRate, systolicBP, temperature, alertness,
           inspiredOxygen, performanceStatus):
    """
    (string, int, float, float, float, float, float, string, string, string) -> float

    Calculate the PRIEST (Pandemic Respiratory Infection Emergency System Triage) Clinical Severity Score
    based on various clinical parameters and determine the corresponding risk of adverse outcome within 30 days.

    This score helps in assessing the severity of respiratory infections in the context of a pandemic.

    Parameters:
    sex (str): Gender assigned at birth ('male' or 'female').
    age (int): Age of the patient in years.
    respiratory_rate (int): Respiratory rate in breaths per minute.
    oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1.
    heart_rate (int): Heart rate in beats per minute.
    systolic_bp (int): Systolic blood pressure in mmHg.
    temperature (float): Temperature in degrees Celsius.
    alertness (str): Alertness status ('alert', 'confused or not alert').
    inspired_oxygen (str): Type of inspired oxygen ('air', 'supplemental oxygen').
    performance_status (str): Patient's performance status.

    Returns:
    float: Risk of adverse outcome within 30 days as a probability between 0 and 1, or None for invalid input.

    Examples:
    >>> priest('female', 80, 31, 0.83, 60, 91, 35.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59

    >>> priest('male', 50, 20, 0.53, 90, 102, 19.0, 'alert', 'air', 'unrestricted normal activity')
    0.29

    >>> priest('male', 30, 20, 0.12, 52, 65, 21.0, 'alert', 'air', 'limited self-care')
    0.46
    """
# Initialize score variable
    priestScore = 0

    # Add sex score
    sex = sex.lower()
    if sex == 'female': priestScore += 0
    elif sex == 'male': priestScore += 1
    else: return None  # Return None if sex is invalid

    # Add age score
    if age < 50: priestScore += 0
    elif 50 <= age <= 65: priestScore += 2
    elif 66 <= age <= 80: priestScore += 3
    else: priestScore += 4

    # Add respiratory rate score
    if respiratoryRate < 9: priestScore += 3
    elif 9 <= respiratoryRate <= 11: priestScore += 1
    elif 12 <= respiratoryRate <= 20: priestScore += 0
    elif 21 <= respiratoryRate <= 24: priestScore += 2
    elif respiratoryRate > 24: priestScore += 3

    # Add oxygen saturation score
    if oxygenSaturation > 0.95: priestScore += 0
    elif 0.94 <= oxygenSaturation <= 0.95: priestScore += 1
    elif 0.92 <= oxygenSaturation <= 0.93: priestScore += 2
    else: priestScore += 3

    # Add heart rate score
    if heartRate < 41: priestScore += 3
    elif 41 <= heartRate <= 50: priestScore += 1
    elif 51 <= heartRate <= 90: priestScore += 0
    elif 91 <= heartRate <= 110: priestScore += 1
    elif 111 <= heartRate <= 130: priestScore += 2
    else: priestScore += 3

    # Add systolic BP score
    if systolicBP < 91: priestScore += 3
    elif 91 <= systolicBP <= 100: priestScore += 2
    elif 101 <= systolicBP <= 110: priestScore += 1
    elif 111 <= systolicBP <= 219: priestScore += 0
    else: priestScore += 3

    # Add temperature score
    if temperature < 35.1: priestScore += 3
    elif 35.1 <= temperature <= 36.0: priestScore += 1
    elif 36.1 <= temperature <= 38.0: priestScore += 0
    elif 38.1 <= temperature <= 39.0: priestScore += 1
    else: priestScore += 2

    # Add alertness score
    alertness = alertness.lower()
    if alertness == 'alert': priestScore += 0
    else: priestScore += 3

    # Add inspired oxygen score
    inspiredOxygen = inspiredOxygen.lower()
    if inspiredOxygen == 'air': priestScore += 0
    elif inspiredOxygen == 'supplemental oxygen': priestScore += 2

    # Add performance status score
    performanceStatus = performanceStatus.lower()
    if performanceStatus == 'unrestricted normal activity': priestScore += 0
    elif performanceStatus == 'limited strenuous activity, can do light activity': priestScore += 1
    elif performanceStatus == 'limited activity, can self-care': priestScore += 2
    elif performanceStatus == 'limited self-care': priestScore += 3
    elif performanceStatus == 'bed/chair bound, no self-care': priestScore += 4

    # Calculate the probability based on score
    if 0 <= priestScore <= 1: return 0.01
    elif 2 <= priestScore <= 3: return 0.02
    elif priestScore == 4: return 0.03
    elif priestScore == 5: return 0.09
    elif priestScore == 6: return 0.15
    elif priestScore == 7: return 0.18
    elif priestScore == 8: return 0.22
    elif priestScore == 9: return 0.26
    elif priestScore == 10: return 0.29
    elif priestScore == 11: return 0.34
    elif priestScore == 12: return 0.38
    elif priestScore == 13: return 0.46
    elif priestScore == 14: return 0.47
    elif priestScore == 15: return 0.49
    elif 16 <= priestScore <= 25: return 0.59
    elif priestScore >= 26: return 0.99
    else: return "Score not found"

In [29]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('female', 80, 31, 0.83, 60, 91, 35.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 50, 20, 0.53, 90, 102, 19.0, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.29
ok
Trying:
    priest('male', 30, 20, 0.12, 52, 65, 21.0, 'alert', 'air', 'limited self-care')
Expecting:
    0.46
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [31]:
pip install requests

In [30]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    (int, string, float) -> String
    Calls a REST web service to determine the appropriate hospital based on age, sex, and risk percentage.

    Parameters:
    - age (int): The age of the patient.
    - sex (str): The sex of the patient ('male' or 'female').
    - risk_pct (float): The risk percentage as a decimal (e.g., 0.1 for 10%).

    Returns:
    - str: The name of the recommended hospital, or None if the request fails.

    >>> find_hospital(83, 'male', 0.9)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(65, 'female', 0.8)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(32, 'male', 0.8)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(20, 'female', 0.53)
    'Emory Dunwoody Medical Center'
    """
    # Constructing the URL
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_pct}"

    try:
        # Sending a GET request to the web service
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parsing the JSON response
            data = response.json()
            # Returning the hospital name
            return data.get('hospital')
        else:
            return None
    except:
        # Handle any errors that occur during the API request
        return None

In [32]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(83, 'male', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(65, 'female', 0.8)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(32, 'male', 0.8)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(20, 'female', 0.53)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [8]:
import requests

def get_address(hospital_name):
    """
    (string) -> String
    Retrieves the address of a given hospital from a remote JSON file.

    Parameters:
    - hospital_name (str): The name of the hospital.

    Returns:
    - str: The address of the hospital, or None if not found.

    Examples:
        >>> get_address("HABERSHAM COUNTY MEDICAL CTR")
        '541 HISTORIC HIGHWAY 441-NORTH'

        >>> get_address("COFFEE REGIONAL MEDICAL CENTER")
        '1101 OCILLA ROAD'

        >>> get_address("CARL VINSON VA MEDICAL CENTER")
        'GENERAL MEDICAL AND SURGICAL HOSPITALS'
    """
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        # Send a GET request to retrieve the JSON data
        response = requests.get(url)
        hospitals = response.json()

        # Search for the hospital by name
        if hospital_name.upper() in hospitals:
            hospital_data = hospitals[hospital_name.upper()]
            return f"{hospital_data['ADDRESS']}"
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

In [9]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("HABERSHAM COUNTY MEDICAL CTR")
Expecting:
    '541 HISTORIC HIGHWAY 441-NORTH'
ok
Trying:
    get_address("COFFEE REGIONAL MEDICAL CENTER")
Expecting:
    '1101 OCILLA ROAD'
ok
Trying:
    get_address("CARL VINSON VA MEDICAL CENTER")
Expecting:
    'GENERAL MEDICAL AND SURGICAL HOSPITALS'
**********************************************************************
File "__main__", line 21, in NoName
Failed example:
    get_address("CARL VINSON VA MEDICAL CENTER")
Expected:
    'GENERAL MEDICAL AND SURGICAL HOSPITALS'
Got:
    '1826 VETERANS BLVD'


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [33]:
import requests

def process_people(url):
    """
()    url) -> list

    Processes patient data from a pipe-delimited file. It determines the appropriate hospital and its address
    based on the risk percentage calculated using the priest function.

    Parameters:
    - url (str): The URL of the pipe-delimited file.

    Returns:
    - list: A list of dictionaries, each containing patient details, their assigned hospital, and address.
    """
    try:
        # Retrieve the file
        response = requests.get(url)
        response.raise_for_status()
        data = response.text

        # Process each line of the file
        results = []
        lines = data.splitlines()
        headers = lines[0].split('|')  # Assuming the first line contains headers
        for line in lines[1:]:
            values = line.split('|')
            patient_info = dict(zip(headers, values))

            # Calculate the risk percentage using the priest function
            risk_pct = priest(
                patient_info['sex'].lower(),
                int(patient_info['age']),
                float(patient_info['breath']),
                float(patient_info['o2 sat']),
                int(patient_info['heart']),
                int(patient_info['systolic bp']),
                float(patient_info['temp']),
                patient_info['alertness'],
                patient_info['inspired'],
                patient_info['status']
            )

            # Determine the hospital based on the calculated risk percentage
            hospital = find_hospital(int(patient_info['age']), patient_info['sex'].lower(), risk_pct)

            # Get the address of the hospital
            address = get_address(hospital) if hospital else "Address not found"

            # Compile the results into a dictionary
            patient_dict = {
                patient_info['patient']: [
                    patient_info['sex'], int(patient_info['age']), float(patient_info['breath']),
                    float(patient_info['o2 sat']), float(patient_info['heart']), int(patient_info['systolic bp']),
                    float(patient_info['temp']), patient_info['alertness'], patient_info['inspired'],
                    patient_info['status'], hospital, address
                ]
            }
            results.append(patient_dict)

        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage (replace 'url' with the actual file URL)
url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
results = process_people(url)
print(results)

[{'E9559': ['FEMALE', 40, 24.0, 0.96, 105.0, 115, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9385': ['Female', 51, 19.0, 0.99, 90.0, 91, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E3067': ['female', 40, 29.0, 0.96, 105.0, 95, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9422': ['FEMALE', 66, 19.0, 0.96, 135.0, 115, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE']}, {'E8661': ['MALE', 18, 22.0, 0.95, 105.0, 115, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW']}, {'E6235': ['Male', 16, 23.0, 0.96, 135.0, 115, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted nor

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [38]:
import requests

# Function to download and load the expected results
def load_expected_results(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to download the expected results")
        return {}

# URL for the expected results
expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

# Load expected results
expected_results = load_expected_results(expected_results_url)

# Assuming `results` is your output from the `process_people` function, and it's formatted as a list of dicts
# Convert your results into a similar format as the expected results for easier comparison
my_results = {list(patient.keys())[0]: list(patient.values())[0] for patient in results}

# Function to compare results, focusing on hospital name and address
def compare_results(expected, actual):
    discrepancies = []
    for patient_id, details in expected.items():
        if patient_id in actual:
            # Extract hospital name and address for comparison
            expected_hospital = details[11]
            expected_address = details[12]
            actual_hospital = actual[patient_id][10]
            actual_address = actual[patient_id][11]

            # Check if there's a mismatch in hospital name or address
            if expected_hospital != actual_hospital or expected_address != actual_address:
                discrepancies.append((patient_id, {
                    'Expected Hospital': expected_hospital,
                    'Actual Hospital': actual_hospital,
                    'Expected Address': expected_address,
                    'Actual Address': actual_address
                }))
        else:
            discrepancies.append((patient_id, {
                'Expected Hospital': details[11],
                'Actual Hospital': 'Missing in actual results',
                'Expected Address': details[12],
                'Actual Address': 'Missing in actual results'
            }))
    return discrepancies

# Perform the comparison
discrepancies = compare_results(expected_results, my_results)

# Print the discrepancies
if discrepancies:
    print("Discrepancies found:")
    for discrepancy in discrepancies:
        print(discrepancy)
else:
    print("All results match perfectly!")

All results match perfectly!


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---